# Problem

Classical machine learning classification uses cross entropy loss. It works well when classes are well-seperated, but struggles in real-world scenarios where:
1. High intra-class variance: Samples from the same class look very different(e.g; the same person photographed from different angles,lightening and angles).
2. Low inter-class variance: Samples from different classes look similar(e.g., two different people who happen to look alike).

Applications like face recognition, fingerprint matching, image retrieval, and  person re-identification all face this challenge.